<a target="_blank" href="https://colab.research.google.com/github/Nick24-hub/Advanced-Topics-in-Neural-Networks-Car-Racing/blob/main/Car_Racing.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
import gymnasium as gym

In [2]:
env = gym.make("CarRacing-v2", continuous=False)
env.reset()

(array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=uint8),
 {})

In [3]:
%%time

total_reward = 0
number_of_episodes = 30

for x in range(number_of_episodes):
    episode_reward = 0
    while(True):
        action = env.action_space.sample()
        observation, reward, terminated, truncated, info = env.step(action)
        episode_reward += reward
        if truncated:
            env.reset()
            break
    total_reward += episode_reward

print(f"Average reward with random actions: {total_reward / number_of_episodes}")

Average reward with random actions: -55.57657052117968
CPU times: total: 3.94 s
Wall time: 3min 17s


In [4]:
env.close()